# Configuring the model, running g-tools and output files and infos

This tutorial shows how to configure the model and how to run Fermipy-LAT g-tools with Fermipy.
Many parts of this tutorial are taken directly from the documentation page of Fermipy: [fermipy.readthedocs](http://fermipy.readthedocs.io/en/latest/). 
I suggest to visit it to find further informations.

# Configuring the model

The model can be configured with a configuration file or directly running Fermipy with your script.

The configuration file defines the data selection and analysis parameters. The configuration file has a hierarchical structure that groups parameters into dictionaries that are keyed to a section name (data, binning, etc.). 

When creating a class instance, the configuration is initialized by passing either a configuration dictionary or configuration file path to the class constructor. 
Keyword arguments can be passed to the constructor to override configuration parameters in the input dictionary. 
In the following example the config dictionary defines values for the parameters emin and emax. By passing a dictionary for the selection keyword argument, the value of emax in the keyword argument (10000) overrides the value of emax in the input dictionary.

The configuration file can be created also with a yaml file.
Below I report a sample of configuration applied for an analysis of the SMC:

The configuration file has the same structure as the configuration dictionary such that one can read/write configurations using the load/dump methods of the yaml module:

DATA

The data section defines the input data files for the analysis (FT1, FT2, and livetime cube). 
evfile and scfile can either be individual files or group of files. The optional ltcube option can be used to choose a pre-generated livetime cube. 
If ltcube is null a livetime cube will be generated at runtime with gtltcube.

Below an example of the data part:

The options for the data component are the following:
* cacheft1: Cache FT1 files when performing binned analysis. If false then only the counts cube is retained.
* evfile: Path to FT1 file or list of FT1 files.
* ltcube: Path to livetime cube. If none a livetime cube will be generated with gtmktime.
* scfile: Path to FT2 (spacecraft) file.

BINNING

Options in the binning section control the spatial and spectral binning of the data. 

The roiwidth is the ROI width, npix specifies the number of pixels, binsz indicates the pixel size, binsperdec is the number of energy bins per decade and projtype is the type of projection and can be WCS and HEALPIX.

Other options are: 
* coordsys: for the coordinate system Galactic or Celestial (CEL or GAL), 
* hpx_order: is the healpix order, hpx_ordering_scheme is the HEALPix Ordering Scheme, 
* proj: is the Spatial projection for WCS mode, Width of the ROI in degrees. 
* roiwidth: is the number of pixels in each spatial dimension will be set from roiwidth / binsz (rounded up).

COMPONENTS

The components section can be used to define analysis configurations for independent subselections of the data. Each subselection will have its own binned likelihood instance that is combined in a global likelihood function for the ROI (implemented with the SummedLikelihood class in pyLikelihood). The components section is optional and when set to null (the default) only a single likelihood component will be created with the parameters of the root analysis configuration.

The component section is defined as a list of dictionaries where each element sets analysis parameters for a different subcomponent of the analysis. The component configurations follow the same structure and accept the same parameters as the root analysis configuration. Parameters not defined in a given element will default to the values set in the root analysis configuration.

The following example illustrates how to define a Front/Back analysis with two components. Files associated to each component will be given a suffix according to their order in the list (e.g. file_00.fits, file_01.fits, etc.).

The following example illustrates how to define an analysis divided in four components. Each of them has a different zmax and uses a different evtype and isotropic template.

EXTENSION

The options in extension control the default behavior of the extension method. For more information about using this method see the Extension Fitting page. The main options are the following:
* fit_ebin: Perform a fit for the angular extension in each analysis energy bin.
* fit_position: Perform a simultaneous fit to the source position and extension.
* fix_shape: Fix spectral shape parameters of the source of interest. If True then only the normalization parameter will be fit.
* free_background: Leave background parameters free when performing the fit. If True then any parameters that are currently free in the model will be fit simultaneously with the source of interest.
* free_radius: Free normalizations of background sources within this angular distance in degrees from the source of interest. If None then no sources will be freed.
* sqrt_ts_threshold: Threshold on sqrt(TS_ext) that will be applied when update is True. If None then nothreshold is applied.
* width: Sequence of values in degrees for the likelihood scan over spatial extension (68% containment radius). If this argument is None then the scan points will be determined from width_min/width_max/width_nstep.
* width_max: Maximum value in degrees for the likelihood scan over spatial extent.
* width_min: Minimum value in degrees for the likelihood scan over spatial extent.
* width_nstep: Number of scan points between width_min and width_max. Scan points will be spaced evenly on a logarithmic scale between width_min and width_max.

There is a notebook dedicated to this analysis.

FILEIO

The fileio section collects options related to file bookkeeping. The outdir option sets the root directory of the analysis instance where all output files will be written. If outdir is null then the output directory will be automatically set to the directory in which the configuration file is located. Enabling the usescratch option will stage all output data files to a temporary scratch directory created under scratchdir.

Sample fileio Configuration

The options for the fileio are the following:
* logfile: Path to log file. If None then log will be written to fermipy.log.
* outdir: Path of the output directory. If none this will default to the directory containing the configuration file.
* outdir_regex: Stage files to the output directory that match at least one of the regular expressions in this list. This option only takes effect when usescratch is True.
* savefits: Save intermediate FITS files.
* scratchdir: Path to the scratch directory. If usescratch is True then a temporary working directory will be created under this directory.
* usescratch: Run analysis in a temporary working directory under scratchdir.
* workdir: Path to the working directory.
* workdir_regex: Stage files to the working directory that match at least one of the regular expressions in this list. This option only takes effect when usescratch is True.

GTLIKE

Options in the gtlike section control the setup of the likelihood analysis include the IRF name (irfs).

A subsample of options for the gtlike section is listed below:
* edisp: Enable the correction for energy dispersion.
* edisp_disable: Provide a list of sources for which the edisp correction should be disabled.
* use_external_srcmap: Use an external precomputed source map file.
* use_scaled_srcmap: Generate source map by scaling an external srcmap file.
* wmap: Likelihood weights map.

The Likelihood wights map is created and used in order to account for example for the uncertainty in the interstellar emission.

MODEL

The model section collects options that control the inclusion of point-source and diffuse components in the model. galdiff and isodiff set the templates for the Galactic IEM and isotropic diffuse respectively. catalogs defines a list of catalogs that will be merged to form a master analysis catalog from which sources will be drawn. Valid entries in this list can be FITS files or XML model files. sources can be used to insert additional point-source or extended components beyond those defined in the master catalog. src_radius and src_roiwidth set the maximum distance from the ROI center at which sources in the master catalog will be included in the ROI model.

Sample for the Model section:

In the sample above the official IEM and isotropic template are used together with the 3FGL catalog given with a fits file and by an additional catalog given with an xml file following the notation of the Fermi Science Tools [fermi.fssc.spectralmodel](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/xml_model_defs.html#xmlModelDefinitions).
Then two sources named as SourceA and SourceB are added.

To define two or more galactic diffuse components you can optionally define the galdiff and isodiff parameters as lists. A separate component will be generated for each element in the list with the name galdiffXX or isodiffXX where XX is an integer position in the list.

To explicitly set the name of a component you can define any element as a dictionary containing name and file fields:

The list of sources for inclusion in the ROI model is set by defining a list of catalogs with the catalogs parameter. Catalog files can be in either XML or FITS format. Sources from the catalogs in this list that satisfy either the src_roiwidth or src_radius selections are added to the ROI model. If a source is defined in multiple catalogs the source definition from the last file in the catalogs list takes precedence.

Fermipy supports four spatial models which are defined with the SpatialModel property:

* PointSource : A point source (SkyDirFunction).
* RadialGaussian : A symmetric 2D Gaussian with width parameter ‘Sigma’.
* RadialDisk : A symmetric 2D Disk with radius ‘Radius’.
* SpatialMap : An arbitrary 2D shape with morphology defined by a FITS template.

The spatial extension of RadialDisk and RadialGaussian can be controlled with the SpatialWidth parameter which sets the 68% containment radius in degrees. Note for ST releases prior to 11-01-01, RadialDisk and RadialGaussian sources will be represented with the SpatialMap type.

OPTIMIZER

This section provides informations for the setup used for the optimization. The list of options are the following:
* init_lambda: Initial value of damping parameter for step size calculation when using the NEWTON fitter. A value of zero disables damping.
* max_iter: Maximum number of iterations for the Newtons method fitter.
* min_fit_quality: Set the minimum fit quality.
* optimizer: Set the optimization algorithm to use when maximizing the likelihood function.
* retries: Set the number of times to retry the fit when the fit quality is less than min_fit_quality.
* tol: Set the optimizer tolerance.
* verbosity

PLOTTING

This section selects the options to be used to make the plots:
* cmap: Set the colormap for 2D plots. Default magma
* cmap_resid: Set the colormap for 2D residual plots.
* figsize: Set the default figure size.
* format: format of images
* interactive: Enable interactive mode. If True then plots will be drawn after each plotting command.
* label_ts_threshold: TS threshold for labeling sources in sky maps. If None then no sources will be labeled.

SELECTION

The selection section collects parameters related to the data selection and target definition. The majority of the parameters in this section are arguments to gtselect and gtmktime. The ROI center can be set with the target parameter by providing the name of a source defined in one of the input catalogs (defined in the model section). Alternatively the ROI center can be defined by giving explicit sky coordinates with ra and dec or glon and glat.

A sample of options for this section are the following:
* emax/emin: Maximum/Minimum Energy (MeV)
* evclass: Event class selection.
* evtype: Event type selection.
* filter: Filter string for gtmktime selection.
* logemax/logemin: Maximum/Minimum Energy (log10(MeV))
* phasemax/phasemin: Maximum/Minimum pulsar phase
* radius: Radius of data selection. If none this will be automatically set from the ROI size.
* target: Choose an object on which to center the ROI. This option takes precendence over ra/dec or glon/glat.
* tmax/tmin: Maximum/Minimum time (MET).
* zmax: Maximum zenith angle.

# Running g-tools

First of all you need to load the configuration file, create the object gta and run the tool gta.setup that implements the ST gtselect, gtmktime, gtbin, gtexpcube, gtsrcmap tools

With the commands below we import the packages needed to run the script.

In [1]:
%matplotlib inline

import os
import numpy as np
from fermipy.gtanalysis import GTAnalysis
from fermipy.plotting import ROIPlotter, SEDPlotter
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import Image

Then we untar the file ../data/SMC_data.tar.gz. This will copy the config.yaml and ft1 file in the notebook directory.

In [4]:
if os.path.isfile('../data/SMC_data.tar.gz'):
    !tar xzf ../data/SMC_data.tar.gz
else:
    !curl -OL https://raw.githubusercontent.com/fermiPy/fermipy-extras/master/data/SMC_data.tar.gz
    !tar xzf SMC_data.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 3523k  100 3523k    0     0  5274k      0 --:--:-- --:--:-- --:--:-- 5275k


In [3]:
gta = GTAnalysis('./SMC_data/config.yaml')
matplotlib.interactive(True)
gta.setup()

2024-06-10 16:00:20 INFO    GTAnalysis.__init__(): 
--------------------------------------------------------------------------------
fermipy version 1.2.2 
ScienceTools version 2.2.0


Exception: Failed to resolve file path: $FERMI_DIFFUSE_DIR/gll_iem_v06.fits

The setup() method performs the data preparation and response calculations needed for the analysis (selecting the data, creating counts and exposure maps, etc.). Depending on the data selection and binning of the analysis this will often be the slowest step in the analysis sequence. The output of setup() is cached in the analysis working directory so subsequent calls to setup() will run much faster.

The g-tools performed with gta.setup() are the following:
* gtselect: [gtselect_fssc](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtselect.txt)
* gtmktime: [gtmiktime_fssc](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtmktime.txt)
* gtbin: [gtbin_fssc](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtbin.txt)
* gtsrcmaps: [gtsrcmaps_fssc](https://fermi.gsfc.nasa.gov/ssc/data/analysis/scitools/help/gtsrcmaps.txt)

In order to have a summary of the source model you can use the command:

In [ ]:
gta.print_model()

The table above contains:
* sourcename as given in the catalog
* offset from the center of the roi. Sources are ranked with respect to this parameter
* normalization of the SED
* energy flux of the source
* index of the SED with a PowerLaw SED
* Test Statistics
* npred is the number of sources associated to the source
* free column that tells if the SED parameter of the sources are free (if the column is filled with an asterix) or fixed without no asterix.

# Source dictionary

First of all, let's make a fit using gta.fit tool.
Source fitting with fermipy is generally performed with the optimize and fit methods. fit is a wrapper on the pyLikelihood fit method and performs a likelihood fit of all free parameters of the model. This method can be used to manually optimize of the model by calling it after freeing one or more source parameters.

In [ ]:
gta.free_sources()
fitresult=gta.fit()

In [ ]:
gta.print_model()

Now you see that the ts column is filled and the free column has all asterix because we have freed the sources.

Fermipy gives the possibility to access many informations on the sources in the model.
Below a few examples.

NAME AND CHARACTERISTICS OF THE SOURCE

In [ ]:
print gta.roi.sources[0]['name'] #name of the source
print gta.roi.sources[0]['Source_Name'] #name of the source
print gta.roi.sources[0]['SpatialModel'] #spatial model
print gta.roi.sources[0]['SpatialWidth'] #spatial size parameter
print gta.roi.sources[0]['SpatialType'] #spatial size parameter
print gta.roi.sources[0]['SourceType'] #Source type
print gta.roi.sources[0]['SpectrumType'] #Spectrum type string
print gta.roi.sources[0]['Spatial_Filename'] #Path to spatial template
print gta.roi.sources[0]['Spectrum_Filename'] #Path to the SED source template
print gta.roi.sources[0]['correlation'] #Dictionary of correlation coefficients.
print gta.roi.sources[0]['model_counts'] #Vector of predicted counts for this source

CHARACTERISTICS OF THE POSITION OF THE SOURCE

In [ ]:
print gta.roi.sources[0]['ra'] #ra
print gta.roi.sources[0]['dec'] #dec
print gta.roi.sources[0]['glon'] #glon
print gta.roi.sources[0]['glat'] #glat
print gta.roi.sources[0]['ra_err'] #error for ra
print gta.roi.sources[0]['dec_err'] #error for dec
print gta.roi.sources[0]['glon_err'] #error for glon
print gta.roi.sources[0]['glat_err'] #error for glat
print gta.roi.sources[0]['pos_err'] #error for the position in deg
print gta.roi.sources[0]['pos_r68'] #68% CL error for the position
print gta.roi.sources[0]['pos_r95'] #95% CL error for the position
print gta.roi.sources[0]['pos_r99'] #99% CL error for the position
print gta.roi.sources[0]['pos_err_semimajor'] #1-sigma uncertainty (deg) along major axis of uncertainty ellipse.
print gta.roi.sources[0]['pos_err_semiminor'] #1-sigma uncertainty (deg) along minor axis of uncertainty ellipse.
print gta.roi.sources[0]['offset_ra'] #Right ascension offset from ROI center in local celestial projection (deg).
print gta.roi.sources[0]['offset_dec'] #Declination offset from ROI center in local celestial projection (deg).
print gta.roi.sources[0]['offset_glon'] #Galactic longitude offset from ROI center in local galactic projection (deg).
print gta.roi.sources[0]['offset_glat'] #Galactic latitude offset from ROI center in local galactic projection (deg).
print gta.roi.sources[0]['offset'] #Angular offset from ROI center (deg).

The error for the position is nan because we have not calculated yet the relocalization. 
Let's run the localization for a source with gta.localize tool.

In [ ]:
gta.free_sources(free=True)
gta.print_model()
gta.free_sources(skydir=gta.roi[gta.roi.sources[2].name].skydir,distance=[3.0],free=True)
gta.print_model()
localsmc = gta.localize(gta.roi.sources[2].name, update=True, make_plots=True)
gta.print_model()

Using the option make_plots=True a few control plots are created like 3fgl_j0023.9-7203_localize_peak.png with the result for the likelihood analysis for the besy fit position and error for the position.

In [ ]:
Image(filename='3fgl_j0023.9-7203_localize_peak.png') 

In [ ]:
print gta.roi.sources[2]['ra'] #ra
print gta.roi.sources[2]['dec'] #dec
print gta.roi.sources[2]['glon'] #glon
print gta.roi.sources[2]['glat'] #glat
print gta.roi.sources[2]['ra_err'] #error for ra
print gta.roi.sources[2]['dec_err'] #error for dec
print gta.roi.sources[2]['glon_err'] #error for glon
print gta.roi.sources[2]['glat_err'] #error for glat
print gta.roi.sources[2]['pos_err'] #error for the position in deg
print gta.roi.sources[2]['pos_r68'] #68% CL error for the position
print gta.roi.sources[2]['pos_r95'] #95% CL error for the position
print gta.roi.sources[2]['pos_r99'] #99% CL error for the position
print gta.roi.sources[2]['pos_err_semimajor'] #1-sigma uncertainty (deg) along major axis of uncertainty ellipse.
print gta.roi.sources[2]['pos_err_semiminor'] #1-sigma uncertainty (deg) along minor axis of uncertainty ellipse.
print gta.roi.sources[2]['offset_ra'] #Right ascension offset from ROI center in local celestial projection (deg).
print gta.roi.sources[2]['offset_dec'] #Declination offset from ROI center in local celestial projection (deg).
print gta.roi.sources[2]['offset_glon'] #Galactic longitude offset from ROI center in local galactic projection (deg).
print gta.roi.sources[2]['offset_glat'] #Galactic latitude offset from ROI center in local galactic projection (deg).
print gta.roi.sources[2]['offset'] #Angular offset from ROI center (deg).

SED PARAMETERS FLUX AND SCAN OF FLUX

In [ ]:
print gta.roi.sources[0]['param_names'] #Names of spectral parameters.
print gta.roi.sources[0]['param_values'] #Spectral parameter values.
print gta.roi.sources[0]['param_errors'] #Spectral parameters errors.
print gta.roi.sources[0]['ts'] #Source test statistic.
print gta.roi.sources[0]['loglike'] #Log-likelihood of the model evaluated at the best-fit normalization of the source.
print gta.roi.sources[0]['flux_scan'] #Flux values for scan of source normalization.
print gta.roi.sources[0]['norm_scan'] #Normalization parameters values for scan of source normalization.
print gta.roi.sources[0]['npred'] #Number of predicted counts from this source integrated over the analysis energy range.
print gta.roi.sources[0]['flux'] #Photon flux (cm−2 s−1) integrated over analysis energy range
print gta.roi.sources[0]['flux_err'] #Photon flux uncertainty (cm−2 s−1) integrated over analysis energy range
print gta.roi.sources[0]['flux_ul95'] #95% CL upper limit on the photon Differential photon flux (cm−2 s−1 MeV−1tegrated over analysis energy range
print gta.roi.sources[0]['dnde'] #Differential photon flux (cm−2 s−1 MeV−1) evaluated at the pivot energy.

In [ ]:
gta.write_roi('model_test',make_plots=True,save_model_map=True)

# Customizing your model

As we saw before you can customize your model directly in the config.yaml file.
However, sometimes you want to change something in your model directly in your Python script.

You can free or fix sources using gta.free_sources.
First let's fix the SED parameter of all the sources.

In [ ]:
gta.free_sources(free=False)
gta.print_model()

Now we free all parameters:

In [ ]:
gta.free_sources(free=True)
gta.print_model()

Now we free all the SED parameters of sources within 3 degrees from 3FGL J0059.0-7242e:

In [ ]:
gta.free_sources(free=False)
gta.free_sources(skydir=gta.roi['3FGL J0059.0-7242e'].skydir,distance=3.0)
gta.print_model()

Now we want to delete the source 3FGL J0021.6-6835:

In [ ]:
gta.delete_source('3FGL J0021.6-6835')
gta.print_model()

You can also delete all the sources as a function of the npred/ts or position using the options minmax_npred/minmax_ts/skydir options. Finally, you can delete sources in a given list using the option names.

In [ ]:
help(gta.delete_sources)

In the example below we delete all the sources that have an npred=[0,500]

In [ ]:
gta.delete_sources(minmax_npred=[0,500])
gta.print_model()

We can aldo add a new source in the model using the gta.add_source function.

In [ ]:
help(gta.add_source)

In the example below we add a Source called Source_PL with a PLSuperExpCutoff and pointlike and a source called Source_Gauss that is spatial extended with a PowerLaw SED and with a RadialGaussian template with an extension of 1 deg.

In [ ]:
gta.add_source('Source_PL',{ 'glon' : 300., 'glat' : -46.,'SpectrumType' : 'PLSuperExpCutoff', 'Index1':-1.5, 'Index2' : 1.0,'Scale' : 1000,'Prefactor':1e-9,'SpatialModel' : 'PointSource' })
gta.add_source('Source_Gauss',{ 'glon' : 302., 'glat' : -45.,'SpectrumType' : 'PowerLaw', 'Index':2.0,'Scale' : 1000,'Prefactor':1e-9,'SpatialModel' : 'RadialGaussian', 'SpatialWidth': 1.0 })
gta.print_model()

Now I load the model saved into model_test to have back the intial model.

In [ ]:
gta.load_roi('model_test')
gta.print_model()

It is also possible to modify the SED parameters using the functions gta.set_norm, gta.set_parameter, gta.set_parameter_bounds, gta.set_parameter_error .

In [ ]:
help(gta.set_norm)
help(gta.set_parameter)

In [ ]:
print gta.roi['3FGL J0059.0-7242e']['param_names']
print gta.roi['3FGL J0059.0-7242e']['param_values']
print gta.roi['3FGL J0059.0-7242e']['param_errors']

In the example below we fix the normalization to 1e-11 and the slope to 2.0.

In [ ]:
gta.set_norm('3FGL J0059.0-7242e',value=1.)
gta.set_parameter('3FGL J0059.0-7242e',par='Index',value=2.0)
print gta.roi['3FGL J0059.0-7242e']['param_names']
print gta.roi['3FGL J0059.0-7242e']['param_values']
print gta.roi['3FGL J0059.0-7242e']['param_errors']

It is also possible to set the SED shape of a source using gta.set_source_spectrum tool.

In [ ]:
help(gta.set_source_spectrum)

In [ ]:
gta.load_roi('model_test')
gta.print_model()
gta.roi['3FGL J0059.0-7242e']['SpectrumType']

In [ ]:
gta.set_source_spectrum('3FGL J0059.0-7242e',spectrum_type='LogParabola')
gta.roi['3FGL J0059.0-7242e']['SpectrumType']

In [ ]:
gta.fit()

In [ ]:
print gta.roi['3FGL J0059.0-7242e']['param_names']
print gta.roi['3FGL J0059.0-7242e']['param_values']
print gta.roi['3FGL J0059.0-7242e']['param_errors']

As you can see above the tool gta.set_source_spectrum has modified the SED shape from PowerLaw to LogParabola.

It is also possible to set the spatial morphology of a source using gta.set_source_morphology

In [ ]:
help(gta.set_source_morphology)

In [ ]:
gta.set_source_morphology(name='3FGL J0029.1-7045',spatial_model='RadialGaussian',spatial_pars={'SpatialWidth': 1.0} )

In [ ]:
gta.print_model()
print gta.roi['3FGL J0029.1-7045']['SpatialType']
print gta.roi['3FGL J0029.1-7045']['SpatialWidth']